In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from huggingface_hub import from_pretrained_keras, push_to_hub_keras
from keras.models import load_model
from PIL import Image

MODEL_PATH = "/Users/skoneru/Downloads/saved_model_rgb_classifier 2"
TEST_FILES_METADATA_CSV = "/Volumes/FD/ai_or_not/data/cifake/test/FAKE_metadata.csv"

REPO_ID = "konerusudhir/ai-or-not-model"

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
model.summary()

In [ ]:
metadata = pd.read_csv(TEST_FILES_METADATA_CSV, skiprows=1)
test_files = metadata.values
test_files = test_files[:10]

# extract prefix path from TEST_FILES_METADATA_CSV
prefx_path = Path(TEST_FILES_METADATA_CSV).parent
for file in test_files:
    file = file[0]
    image = Image.open(
        os.path.join(prefx_path, file),
    )
    # Convert to RGB
    if image.mode != "RGB":
        image = image.convert("RGB")

    # normalize to 0-1
    image = np.array(image) / 255.0

    resized_image = tf.image.resize_with_pad(image, 224, 224)

    print(f"File: {file} Image shape: {resized_image.shape}")
    predictions = model.predict(np.array([resized_image]))
    # take the argmax of the predictions
    max_index = np.argmax(predictions)

    print(f"Prediction: {predictions} Max: {max_index}")

In [ ]:
# login to huggingface
# hf-cli login

In [ ]:
# Upload model to huggingface
push_to_hub_keras(
    model,
    repo_id=REPO_ID,
    tags=["0.1.1", "classification"],
    branch="main",
)

In [ ]:
model = from_pretrained_keras(REPO_ID, token=True, force_download=True)

In [ ]:
prefx_path = Path(TEST_FILES_METADATA_CSV).parent
for file in test_files:
    file = file[0]
    image = Image.open(
        os.path.join(prefx_path, file),
    )
    # Convert to RGB
    if image.mode != "RGB":
        image = image.convert("RGB")

    # normalize to 0-1
    image = np.array(image) / 255.0

    resized_image = tf.image.resize_with_pad(image, 224, 224)

    print(f"File: {file} Image shape: {resized_image.shape}")
    predictions = model(np.array([resized_image]))
    # take the argmax of the predictions
    max_index = np.argmax(predictions)

    print(f"Prediction: {predictions} Max: {max_index}")